In [11]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.compat.v2.keras.wrappers.scikit_learn import KerasClassifier 

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [3]:
train_data = pd.read_csv("../train_data.csv")
dev_data = pd.read_csv("../dev_data.csv")

train_data = train_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
train_data["is_retweet"] = 1*train_data["is_retweet"]
train_data["is_modified_retweet"] = 1*train_data["is_modified_retweet"]

dev_data = dev_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
dev_data["is_retweet"] = 1*dev_data["is_retweet"]
dev_data["is_modified_retweet"] = 1*dev_data["is_modified_retweet"]

In [4]:
sc = train_data.groupby("author_Id").count().sort_values("tweet",ascending = False)#.head()
set_y = list(sc.index)

In [5]:
def id_to_int_cat(id_ind):
    return np.argwhere(train_data["author_Id"].values[id_ind]==set_y)[0][0]
temp = [id_to_int_cat(i) for i in range(len(train_data["author_Id"]))]
y_train = to_categorical(temp,dtype=int)

In [6]:
X_train = train_data.drop(['tweet','author_Id'],axis=1).values
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train = to_categorical(temp,dtype=int)

In [7]:
def make_ff(n_nodes):
    model = Sequential()
    model.add(Dense(n_nodes,activation = "relu",input_dim=X_train.shape[1],kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.3))
    # model.add(Dense(5000,activation = "relu",kernel_regularizer=regularizers.l2(0.01)))
    # model.add(Dropout(0.3))
    model.add(Dense(y_train.shape[1],activation="softmax"))
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'],)
    
    return model

In [ ]:
filepath = "modelgrid.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop =  EarlyStopping(monitor='val_loss',patience=3, min_delta=0.0001)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="loggrid")
callbacks_list = [checkpoint,tensorboard_callback]#,early_stop,]

n_nodes = [10,20,30]
params = {"n_nodes":n_nodes}
modelw = KerasClassifier(build_fn=make_ff)
clf = GridSearchCV(modelw,params,n_jobs=1)
clf.fit(X_train, y_train)

/anaconda3/envs/TF/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
W0906 18:27:03.043077 4690908608 deprecation.py:323] From /anaconda3/envs/TF/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 164466 samples
82233/82233 [==============================] - 21s 249us/sample - loss: 8.9623 - accuracy: 0.0016
Train on 164466 samples
82233/82233 [==============================] - 21s 257us/sample - loss: 8.9654 - accuracy: 0.0014
Train on 164466 samples
82233/82233 [==============================] - 20s 240us/sample - loss: 8.9589 - accuracy: 0.0015
Train on 164466 samples
82233/82233 [==============================] - 19s 234us/sample - loss: 8.9455 - accuracy: 0.0017
Train on 164466 samples
82233/82233 [==============================] - 21s 257us/sample - loss: 8.9536 - accuracy: 0.0015
Train on 164466 samples
82233/82233 [==============================] - 24s 286us/sample - loss: 8.9456 - accuracy: 0.0015
Train on 164466 samples
82233/82233 [==============================] - 19s 229us/sample - loss: 8.9455 - accuracy: 0.0014
